In [ ]:
#-------------------------------------------------------------------------------
# ADD THE NAMES OF THE TEAM MEMBERS HERE

# Casaburi Angelo - 0622700837
# Carusone Nicola - 0622700814
# Cavallaro Armando - 0622700838
# Cerino Mario - 0622701191
# Siano Giuseppe - 0622700866

#-------------------------------------------------------------------------------

In [ ]:
import os
import tensorflow as tf
import keras
import keras.backend as K
from keras.layers import Input, Dense, Conv2D, Dropout, Flatten, MaxPool2D, concatenate
from keras.models import Model, load_model
import numpy as np
import h5py
import PIL

#-------------------------------------------------------------------------------
# ADD IMPORTS HERE

from tqdm import tqdm

#-------------------------------------------------------------------------------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.chdir('/content/drive/My Drive/Machine Learning/Contest2019/AgeComparison')

In [ ]:
# LOAD TEST SET
f = h5py.File('test_set.h5', 'r')


#-------------------------------------------------------------------------------
# ADD PATH VARIABLES DEFINITION HERE
model_path = 'siamese_model.hdf5'


#-------------------------------------------------------------------------------

In [ ]:
# ADD HERE THE CODE FOR INSTALLING PACKAGES, IF NEEDED
# E.g. !pip install opencv-python

In [ ]:
def load_full_model():
  
  #-----------------------------------------------------------------------------
  # LOAD MODEL HERE

  model = load_model(model_path, custom_objects={'tf': tf})
  # If multiple models are needed use a list
  # model1 = ...
  # model2 = ...
  # model3 = ...
  # model = [model1, model2, model3]


  #-----------------------------------------------------------------------------

  return model

In [ ]:
# LOAD MODEL
model = load_full_model()

# PREDICT FUNCTION
def predict(a,b):
  global model

  #-----------------------------------------------------------------------------
  # OPTIONAL PREPROCESSING HERE

  inp1 = a/255
  inp2 = b/255
  
  #-----------------------------------------------------------------------------

  y_pred = model.predict([inp1,inp2])
  # If multiple models are used, you may need to replace the previous
  # code:
  #     intermediate_value = model[0].predict([inp1,inp2])
  #     y_pred = model[1].predict(intermediate_value)   

  #-----------------------------------------------------------------------------
  # OPTIONAL POSTROCESSING HERE
  ones = np.argmax(y_pred, axis=1)
  for i in range(len(y_pred)):
    y_pred[i][:] = 0
    y_pred[i][ones[i]] = 1

  #-----------------------------------------------------------------------------
  
  return y_pred

# You must not modify the following cells

In [ ]:
# BATCHES GENERATOR

class H5Generator(keras.utils.Sequence):

    def __init__(self, X1 ,X2, y, batch_size):
        self.X1 = X1
        self.X2 = X2
        self.y = y
        self.batch_size = batch_size
        self._l = self.X1.shape[0] // batch_size
        if self.X1.shape[0] % batch_size != 0:
            self._l += 1

    def __len__(self):
        return self._l

    def __getitem__(self, index):
        bs = self.batch_size

        batch_x1 = np.array(
            self.X1[index*bs:min((index+1)*bs, self.X1.shape[0])])
        batch_x2 = np.array(
            self.X2[index*bs:min((index+1)*bs, self.X2.shape[0])])
        batch_y = np.array(
            self.y[index*bs:min((index+1)*bs, self.y.shape[0])])

        return batch_x1, batch_x2, batch_y

In [ ]:
# SCORE
y_pred = None
y_true = None

for x1,x2,y in tqdm(H5Generator(f['X1'],f['X2'], f['Y'], 1000)):
  y_p = predict(x1,x2)

  y_pred = np.append(y_pred,y_p,axis=0) if y_pred is not None else y_p
  y_true = np.append(y_true,y,axis=0) if y_true is not None else y

print("\nSCORE: ",1.0-abs(y_true - y_pred).max(axis=1).mean())